In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -----------------------------
# 1. Load Data
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/GOOGLE COLAB/Dataset/Claims_Denial_Dataset.csv")

# Drop rows where ClaimStatus is missing (target variable)
df = df.dropna(subset=["ClaimStatus"])

# -----------------------------
# 2. Encode categorical variables
# -----------------------------
cat_cols = df.select_dtypes(include=["object"]).columns

encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# -----------------------------
# 3. Define features & target
# -----------------------------
X = df.drop("ClaimStatus", axis=1)
y = df["ClaimStatus"]

# -----------------------------
# 4. Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# -----------------------------
# 5. Scale numeric features
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# 6. Train model
# -----------------------------
model = LogisticRegression(max_iter=500, solver="lbfgs")
model.fit(X_train_scaled, y_train)

# -----------------------------
# 7. Evaluate
# -----------------------------
y_pred = model.predict(X_test_scaled)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))

# -----------------------------
# 8. Predict probabilities
# -----------------------------
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# -----------------------------
# 9. Build final output table
# -----------------------------
# Take original (non-encoded) values for readability
X_test_original = df.loc[X_test.index].copy()

results = pd.DataFrame({
    "ClaimID": X_test_original["ClaimID"],
    "Provider": X_test_original["ProviderID"],
    "Payer": X_test_original["ProviderLocation"],   # assuming this column is Payer
    "CPT": X_test_original["ProcedureCode"],
    "Predicted Risk": y_proba.round(2),
    "Status": ["⚠️ High Risk" if p >= 0.5 else "✅ Low Risk" for p in y_proba]
})

print("\n--- Sample Predictions ---\n")
print(results.head(10))


✅ Accuracy: 0.34296296296296297

--- Sample Predictions ---

      ClaimID  Provider  Payer   CPT  Predicted Risk      Status
1109     2863      3333   1661  3667            0.32  ✅ Low Risk
770      1122       248    602  1207            0.33  ✅ Low Risk
1540     2504      1873   2378  3389            0.30  ✅ Low Risk
4171      330      4492   2967  4362            0.34  ✅ Low Risk
1136     1760      1123   1886  3317            0.33  ✅ Low Risk
119      1539       467   2001  1204            0.36  ✅ Low Risk
858      4256      4235     81  1654            0.29  ✅ Low Risk
4122     1121      2862    442  2371            0.37  ✅ Low Risk
287      1067      2807   1912   474            0.36  ✅ Low Risk
2685     3867       678   2544  1840            0.32  ✅ Low Risk
